In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
# Configuração do lookback (número de dias usados como entrada para prever o próximo dia)
lookback = 30  # 30 dias de histórico para prever o próximo dia

In [3]:
# Função para preparar os dados
def prepare_data(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i])  # Sequência de 'lookback' dias
        y.append(data[i, -1])  # Coluna alvo (wind_avg)
    return np.array(X), np.array(y)

In [5]:
# Carregar os dados
data = pd.read_csv('weather_sum_combinado.csv')

In [ ]:
data = data[['rain_max', 'rad_max', 'temp_avg', 'temp_max', 'temp_min', 'hum_max', 'hum_min', 'wind_max', 'wind_avg']]

In [ ]:
# Remover valores nulos e normalizar
data = data.dropna()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
# Preparar os dados para modelos de séries temporais
X, y = prepare_data(data_scaled, lookback)
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))  # Formato (amostras, timesteps, features)

In [ ]:
# Divisão temporal usando TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# Visualizar os folds
folds_info = []
plt.figure(figsize=(12, 6))
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    folds_info.append((train_index, test_index))
    plt.plot(train_index, [fold + 1] * len(train_index), 'b.', label="Treino" if fold == 0 else "")
    plt.plot(test_index, [fold + 1] * len(test_index), 'r.', label="Teste" if fold == 0 else "")

plt.xlabel("Índice")
plt.ylabel("Fold")
plt.title("Separação dos Folds - TimeSeriesSplit")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Funções para criar os modelos
def create_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(200, input_shape=input_shape))
    model.add(Dense(1))  # Saída
    return model

def create_bilstm(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(200), input_shape=input_shape))
    model.add(Dense(1))
    return model

def create_clstm(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(LSTM(200))
    model.add(Dense(1))
    return model

In [ ]:
# Configurar resultados
results = {'LSTM': [], 'BiLSTM': [], 'CLSTM': []}

In [ ]:
# Treinamento e avaliação dos modelos
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for model_name, create_model in zip(['LSTM', 'BiLSTM', 'CLSTM'], [create_lstm, create_bilstm, create_clstm]):
        print(f"Treinando modelo {model_name}...")
        
        model = create_model((X_train.shape[1], X_train.shape[2]))
        optimizer = Adam(learning_rate=0.005)  # Taxa de aprendizado ajustada
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])
        
        model.fit(X_train, y_train, epochs=250, batch_size=700, verbose=0)
        y_pred = model.predict(X_test).flatten()
        
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        results[model_name].append({'MSE': mse, 'RMSE': rmse, 'R2': r2, 'y_test': y_test, 'y_pred': y_pred})
        print(f"{model_name}: MSE={mse:.4f}, RMSE={rmse:.4f}, R2={r2:.4f}")

In [ ]:
# Comparação de resultados
for model_name, metrics in results.items():
    mse_vals = [m['MSE'] for m in metrics]
    rmse_vals = [m['RMSE'] for m in metrics]
    r2_vals = [m['R2'] for m in metrics]
    print(f"\nModelo: {model_name}")
    print(f"MSE Médio: {np.mean(mse_vals):.4f} ± {np.std(mse_vals):.4f}")
    print(f"RMSE Médio: {np.mean(rmse_vals):.4f} ± {np.std(rmse_vals):.4f}")
    print(f"R² Médio: {np.mean(r2_vals):.4f} ± {np.std(r2_vals):.4f}")

In [ ]:
# Gráficos de Previsão
for model_name, metrics in results.items():
    plt.figure(figsize=(10, 5))
    for fold, m in enumerate(metrics):
        plt.plot(m['y_test'], label=f'Real Fold {fold+1}', alpha=0.5)
        plt.plot(m['y_pred'], label=f'Previsto Fold {fold+1}', alpha=0.5)
    plt.title(f"Real vs Previsto - {model_name}")
    plt.xlabel("Amostras")
    plt.ylabel("Velocidade Média do Vento")
    plt.legend()
    plt.show()
